In [6]:
import ASG
import importlib
import Utility
import re
# importlib.reload(Utility)

In [7]:
# Only need to change the path variable
# path = './CCOE/Xorddos/0aefb67c01a24d05351b093455203fa2.bin'
# path = '../../C ASG/trace/Xorddos/0aefb67c01a24d05351b093455203fa2.bin' 
path = '../../C ASG/trace/Dofloo/0046a78514a658b9b7a4e29e8581f85b.bin'
graph = ASG.AttackGraph(path)
graph.create()

In [9]:
g = graph.draw(False)
file_name = "xorddos"
# g.render("./output-img/" + "test" + "/" + file_name, format="svg")
# g.render("./" + file_name + "/" + file_name, format="svg")

Node before reduction:  65
Nodes after reduction:  65


# For file regex

In [10]:
set_of_objects_file = Utility.get_set_of_objects_file(graph)

# file rule dict
RULES_DICT = Utility.build_RULES_DICT() 

# use the rule dict to build up the regex, it may not match all objects
regex_match_file, regex_non_match_file = Utility.build_file_regex(set_of_objects_file, RULES_DICT) 
file_regex = list(regex_match_file.keys())

print("Objects num:", len(set_of_objects_file))
print("Match Regex num:", len(regex_match_file))
print("Non match file num:", len(regex_non_match_file))

Objects num: 20
Match Regex num: 11
Non match file num: 0


In [11]:
file_regex

['/.*bin/sed',
 '/dev/urandom',
 '/etc/init\\.d/.*',
 '/etc/rc.*\\.d/.*',
 '/etc/rc\\.local',
 '/etc/sed[0-9a-zA-Z]{6}',
 '.*/selinux',
 '/proc/net/dev',
 '/proc/self',
 '/proc/stat',
 '/sys/devices/.*']

In [12]:
regex_match_file

{'/.*bin/sed': ['/bin/sed',
  '/sbin/sed',
  '/usr/bin/sed',
  '/usr/local/bin/sed',
  '/usr/local/sbin/sed',
  '/usr/sbin/sed'],
 '/dev/urandom': ['/dev/urandom'],
 '/etc/init\\.d/.*': ['/etc/init.d/boot.local'],
 '/etc/rc.*\\.d/.*': ['/etc/rc.d/rc.local'],
 '/etc/rc\\.local': ['/etc/rc.local'],
 '/etc/sed[0-9a-zA-Z]{6}': ['/etc/sedQUGLbs',
  '/etc/sedQhw17q',
  '/etc/sedvTqQwq'],
 '.*/selinux': ['/etc/selinux/config', '/selinux', '/sys/fs/selinux'],
 '/proc/net/dev': ['/proc/net/dev'],
 '/proc/self': ['/proc/self/exe'],
 '/proc/stat': ['/proc/stat'],
 '/sys/devices/.*': ['/sys/devices/system/cpu/online']}

In [13]:
regex_non_match_file

[]

# For remain regex

In [14]:
regex_match_proc = Utility.get_regex_match_proc(graph)
regex_match_net = Utility.get_regex_match_net(graph)
regex_match_mem = Utility.get_regex_match_mem(graph)
regex_match_ID = Utility.get_regex_match_ID(graph)
regex_match_permission = Utility.get_regex_match_permission(graph)

In [15]:
regex_match_proc

{'^uname$': ['uname'], '^sh$': ['sh'], '^sed$': ['sed']}

In [16]:
regex_match_net

{'^eth.*': ['NIC'],
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}': ['23.236.66.13:50050'],
 '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+': ['23.236.66.13:50050']}

In [17]:
regex_match_mem

{'0x[0-9a-zA-Z]{8}': ['Memory Address']}

In [18]:
regex_match_ID

{'^UID.*': ['UID:0'],
 '^GID.*': ['GID:0'],
 '^PID.*': ['PID:1514',
  'PID:1513',
  'PID:1515',
  'PID:1516',
  'PID:1518',
  'PID:1519',
  'PID:1520',
  'PID:1521',
  'PID:1522',
  'PID:1524']}

In [19]:
regex_match_permission

{'Permission': ['Permission:0700', 'Permission:022'],
 'Permission:[0-9]{3}': ['Permission:0700', 'Permission:022']}

# Sorted unique step list

In [20]:
sorted_uni_step = Utility.get_sorted_uni_step(graph)
for i, v in enumerate(sorted_uni_step):
    print("Step", i+1, v)
    break

Step 1 ['malware', 'uname', 'exec']


# check all object

In [21]:
# This contain all object including "subject" and "object"
len(Utility.get_set_of_dict(graph)["File"])

22

In [22]:
# Utility.get_set_of_dict(graph)#.keys()

In [23]:
### reduction 1 sink node: use the graph before reduction ### 
dup_step = len(graph.step_list)
print("dup_step:", dup_step)
Utility.get_reduction_statistic(graph)

dup_step: 7577
mem: 1 -> 1 sink: 0
time: 1 -> 1 sink: 0
NIC: 0 -> 1 sink: -1


# Function for retrieving origin_step
- input: a operation pair `<syscall(), regex>`
- output: a list of related asg steps `<step_num, subject, syscall(), object>`, in step order.
    - format: `<101. 1882 -> read() -> /var/run/.lihtpid>`
    - spaces: step:4, subject:7, syscall:10

```
total len of steplist: 284
['  65. /etc/sedQUGLbs -> rename()   -> /etc/rc.local',
 ' 141. /etc/sedQhw17q -> rename()   -> /etc/rc.local',
 ' 184. /etc/sedvTqQwq -> rename()   -> /etc/rc.local']
 ```

In [5]:
class Step:
    '''Store an asg step's data'''
    def __init__(self, number, subject, syscall, object) -> None:
        self.num = number
        self.sub = subject
        self.obj = object
        self.call = syscall
        pass
    def callb(self) -> str:
        '''system call with braces'''
        return f"{self.call}()"
    def __eq__(self, other: object) -> bool:
        '''note: self.num will not be compare (這個方法目前用不到)'''
        if isinstance(other, Step):
            return self.sub == other.sub and self.obj == other.obj and self.call == other.call
        return False
    def __hash__(self):
        return hash((self.sub, self.obj, self.call))

def query_origin_steplist(graph:ASG.AttackGraph, syscall:str, regex:str) -> list[str]:
    '''Query the origin asg steps (triplet), return in a list of formatted string.\n
    Input:
        @graph: ASG.AttackGraph
        @syscall: system call
        @regex: the regex of object
    Ouput example:
        - query `get_origin_steplist(graph, 'rename', '/etc/rc\.local')` will get:
        - [`'  65. /etc/sedQUGLbs -> rename()   -> /etc/rc.local'`, `' 141. /etc/sedQhw17q -> rename()   -> /etc/rc.local'`]
    '''
    # take the first appearence step (unique) and set the step number
    steplist = Utility.get_sorted_uni_step(graph)
    steplist = [Step(i+1, step[0], step[2], step[1]) for i,step in enumerate(steplist)]
    # print('total len of steplist:', len(steplist)) #  284 for dofloo

    # filter object and syscall with input regex
    steplist = [step for step in steplist if re.search(regex, step.obj, re.IGNORECASE)] 
    steplist = [step for step in steplist if step.call == syscall]
    # turn object in to string
    steplist = [f"{step.num:4}. {step.sub:<7} -> {step.callb():10} -> {step.obj}" for step in steplist]
    return steplist

query_origin_steplist(graph, 'rename', '/etc/rc\.local') # step 64, 140, 183
# query_origin_steplist(graph, 'connect', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}') # step 270

total len of steplist: 284


['  65. /etc/sedQUGLbs -> rename()   -> /etc/rc.local',
 ' 141. /etc/sedQhw17q -> rename()   -> /etc/rc.local',
 ' 184. /etc/sedvTqQwq -> rename()   -> /etc/rc.local']

In [5]:
g = graph.draw(True)
g.render(f"dofloo.tmp", format='svg')
del g

Node before reduction:  74
Nodes after reduction:  74
